# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas, string, re
from sklearn.model_selection import train_test_split, GridSearchCV
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from nltk.stem.porter import PorterStemmer
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from nltk.stem.wordnet import WordNetLemmatizer
import pickle

sw= stopwords.words('english')

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pandas.read_sql('InsertTableName', con=engine)
X = df["message"]
Y = df.drop(["message", "original","genre", 'id'], axis = 1)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26216 entries, 0 to 26215
Data columns (total 40 columns):
id                        26216 non-null int64
message                   26216 non-null object
original                  10170 non-null object
genre                     26216 non-null object
related                   26216 non-null int64
request                   26216 non-null int64
offer                     26216 non-null int64
aid_related               26216 non-null int64
medical_help              26216 non-null int64
medical_products          26216 non-null int64
search_and_rescue         26216 non-null int64
security                  26216 non-null int64
military                  26216 non-null int64
child_alone               26216 non-null int64
water                     26216 non-null int64
food                      26216 non-null int64
shelter                   26216 non-null int64
clothing                  26216 non-null int64
money                     26216 non-null i

In [4]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [6]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    
    '''
    This function tokenizes text by filtering out only those characters which are upper and lower characters
    
    Words not in stopwords are lemmatized and returned
    '''
    text =re.sub(r'[^a-zA-Z0-9]'," "  , text)
    text = text.lower()
    text = text.split()

    
    text   = [word for word in text if not word in sw]
    lemmed_text = [WordNetLemmatizer().lemmatize(word) for word in text]
    return lemmed_text
    
    
def tokenize2(text):
    """ This function tokenizes text by filering out the punctuation characters
    
    Words not in stopwords are lemmatized and returned """
    
    text= [char  if char not in  string.punctuation else " " for char in text.lower()]
    text="".join(text)
    #split text into individual words
    text = text.split()
    # Remove stop words  
    
    text  = [word for word in text if word not in sw]
    lemmed_text = [WordNetLemmatizer().lemmatize(word) for word in text]
    
    return lemmed_text


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline(
    [ ("corpus",CountVectorizer(analyzer=tokenize)),
     ("tdif", TfidfTransformer()),
     ("classifier", MultiOutputClassifier( MultinomialNB()))
    ]
)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = .25)
print(X_train.shape, Y_train.shape )


(19662,) (19662, 36)


In [10]:
pipeline.fit(X_train, Y_train)

c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


Pipeline(memory=None,
     steps=[('corpus', CountVectorizer(analyzer=<function tokenize at 0x09402300>, binary=False,
        decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preprocessor=None,
 ...fier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
def make_predictions(classifier, X_test_set, Y_test_set):
    """
    This function automates the output format for displaying the results for a trained model, classifier using the 
    """
    predictions  = classifier.predict(X_test_set)
    predictions = pandas.DataFrame(predictions, columns= Y_test_set.columns)
    for y in Y_test_set.columns:
        print("\n\n" +y + "  \n")
        
        y_true = Y_test[y]
        y_pred = predictions[y]
        
        cr = classification_report(y_true, y_pred, output_dict=True)
        
        if cr.get('1'):
            print("Precision: {:.5}, Recall: {:.5},  F1 score : {:.5} ,Accuracy:  {:.5}" 
               .format(cr['1']["precision"], cr["1"]["recall"], cr["1"]["f1-score"], accuracy_score(y_true, y_pred)) )
        else:
            print("Precision: {}, Recall: {},  F1 score : {} ,Accuracy:  {}" 
               .format(1.0, 1.0, 1.0, accuracy_score(y_true, y_pred)) )
        
       
        
        
        print(confusion_matrix(Y_test_set[y], predictions[y]))
        print()
        print("#" *20 )
    

In [12]:
predictions = pipeline.predict(X_test)

In [13]:
predictions.shape

(6554, 36)

In [14]:
predictions = pandas.DataFrame(predictions, columns= Y.columns)
predictions.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
Y_test.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
8560,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17008,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
621,1,1,0,1,0,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0
23951,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22342,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
for y in Y.columns:
    print("\n" +y + "  \n")
    print(classification_report(Y_test[y],predictions[y]))
    print(confusion_matrix(Y_test[y], predictions[y]))
    print("Accuracy:  {}" .format (accuracy_score(Y_test[y], predictions[y])))
    print("#" *20 )
    #print("The f1 score for category {} is {}".format(y, f1_score(Y_test, predictions[y])) )
    


related  

              precision    recall  f1-score   support

           0       0.81      0.09      0.16      1619
           1       0.77      0.99      0.87      4935

   micro avg       0.77      0.77      0.77      6554
   macro avg       0.79      0.54      0.51      6554
weighted avg       0.78      0.77      0.69      6554

[[ 140 1479]
 [  32 4903]]
Accuracy:  0.7694537686908758
####################

request  

              precision    recall  f1-score   support

           0       0.86      0.99      0.92      5453
           1       0.88      0.23      0.36      1101

   micro avg       0.87      0.87      0.87      6554
   macro avg       0.87      0.61      0.64      6554
weighted avg       0.87      0.87      0.83      6554

[[5420   33]
 [ 850  251]]
Accuracy:  0.8652731156545621
####################

offer  

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00    

c:\python\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[3274  597]
 [ 974 1709]]
Accuracy:  0.7602990540128166
####################

medical_help  

              precision    recall  f1-score   support

           0       0.92      1.00      0.96      6021
           1       0.33      0.00      0.00       533

   micro avg       0.92      0.92      0.92      6554
   macro avg       0.63      0.50      0.48      6554
weighted avg       0.87      0.92      0.88      6554

[[6019    2]
 [ 532    1]]
Accuracy:  0.9185230393652731
####################

medical_products  

              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6237
           1       1.00      0.00      0.01       317

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.98      0.50      0.49      6554
weighted avg       0.95      0.95      0.93      6554

[[6237    0]
 [ 316    1]]
Accuracy:  0.9517851693622216
####################

search_and_rescue  

              precision    recall  f1-score   suppor

Accuracy:  0.9867256637168141
####################

shops  

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        27

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

[[6527    0]
 [  27    0]]
Accuracy:  0.9958803783948733
####################

aid_centers  

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6483
           1       0.00      0.00      0.00        71

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

[[6483    0]
 [  71    0]]
Accuracy:  0.9891669209642966
####################

other_infrastructure  

              precision    recall  f1-score   support

           0       0.95      1.00

In [17]:
classification_report(Y_test[y],predictions[y],  output_dict=True )["1"]

{'f1-score': 0.28914057295136575,
 'precision': 0.8127340823970037,
 'recall': 0.17585089141004862,
 'support': 1234}

In [18]:
make_predictions(pipeline, X_test, Y_test)



related  

Precision: 0.76825, Recall: 0.99352,  F1 score : 0.86648 ,Accuracy:  0.76945
[[ 140 1479]
 [  32 4903]]

####################


request  

Precision: 0.8838, Recall: 0.22797,  F1 score : 0.36245 ,Accuracy:  0.86527
[[5420   33]
 [ 850  251]]

####################


offer  



c:\python\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: 0.0, Recall: 0.0,  F1 score : 0.0 ,Accuracy:  0.99603
[[6528    0]
 [  26    0]]

####################


aid_related  

Precision: 0.74111, Recall: 0.63697,  F1 score : 0.68511 ,Accuracy:  0.7603
[[3274  597]
 [ 974 1709]]

####################


medical_help  

Precision: 0.33333, Recall: 0.0018762,  F1 score : 0.0037313 ,Accuracy:  0.91852
[[6019    2]
 [ 532    1]]

####################


medical_products  

Precision: 1.0, Recall: 0.0031546,  F1 score : 0.0062893 ,Accuracy:  0.95179
[[6237    0]
 [ 316    1]]

####################


search_and_rescue  

Precision: 0.0, Recall: 0.0,  F1 score : 0.0 ,Accuracy:  0.97177
[[6369    0]
 [ 185    0]]

####################


security  

Precision: 0.0, Recall: 0.0,  F1 score : 0.0 ,Accuracy:  0.98184
[[6435    0]
 [ 119    0]]

####################


military  

Precision: 0.0, Recall: 0.0,  F1 score : 0.0 ,Accuracy:  0.96689
[[6337    0]
 [ 217    0]]

####################


child_alone  

Precision: 1.0, Recall: 1.0,  F1 score

### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
parameters = {"classifier__estimator__alpha": [0.1, 0.5, 0.8, 1.0 ],
              "tdif__norm" :["l1", "l2", None],
              "tdif__use_idf": [True, False],
              "corpus__analyzer":[tokenize, tokenize2],
              "corpus__max_features":[1000, 1500, None]}

cv = GridSearchCV(pipeline, parameters, verbose=1, cv=4)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
cv.fit(X_train, Y_train)

Fitting 4 folds for each of 144 candidates, totalling 576 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.c

c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklear

c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklear

c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklear

...

c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklear

c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklear

c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklear

c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklear

GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('corpus', CountVectorizer(analyzer=<function tokenize at 0x09402300>, binary=False,
        decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preprocessor=None,
 ...fier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'corpus__max_features': [1000, 1500, None], 'corpus__analyzer': [<function tokenize at 0x09402300>, <function tokenize2 at 0x094023D8>], 'tdif__norm': ['l1', 'l2', None], 'tdif__use_idf': [True, False], 'classifier__estimator__alpha': [0.1, 0.5, 0.8, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [21]:
make_predictions(cv, X_test, Y_test)



related  

Precision: 0.82082, Recall: 0.95238,  F1 score : 0.88172 ,Accuracy:  0.8076
[[ 593 1026]
 [ 235 4700]]

####################


request  

Precision: 0.7354, Recall: 0.58311,  F1 score : 0.65046 ,Accuracy:  0.89472
[[5222  231]
 [ 459  642]]

####################


offer  

Precision: 0.0, Recall: 0.0,  F1 score : 0.0 ,Accuracy:  0.99603
[[6528    0]
 [  26    0]]

####################


aid_related  

Precision: 0.73163, Recall: 0.66045,  F1 score : 0.69422 ,Accuracy:  0.76182


c:\python\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[3221  650]
 [ 911 1772]]

####################


medical_help  

Precision: 0.57143, Recall: 0.13508,  F1 score : 0.21851 ,Accuracy:  0.92142
[[5967   54]
 [ 461   72]]

####################


medical_products  

Precision: 0.65517, Recall: 0.17981,  F1 score : 0.28218 ,Accuracy:  0.95575
[[6207   30]
 [ 260   57]]

####################


search_and_rescue  

Precision: 1.0, Recall: 0.021622,  F1 score : 0.042328 ,Accuracy:  0.97238
[[6369    0]
 [ 181    4]]

####################


security  

Precision: 1.0, Recall: 0.0084034,  F1 score : 0.016667 ,Accuracy:  0.982
[[6435    0]
 [ 118    1]]

####################


military  

Precision: 0.64286, Recall: 0.20737,  F1 score : 0.31359 ,Accuracy:  0.96994
[[6312   25]
 [ 172   45]]

####################


child_alone  

Precision: 1.0, Recall: 1.0,  F1 score : 1.0 ,Accuracy:  1.0
[[6554]]

####################


water  

Precision: 0.79208, Recall: 0.18692,  F1 score : 0.30246 ,Accuracy:  0.9437
[[6105   21]
 [ 348   80]]

###########

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [22]:
X_genre = df[["genre","message"]]
#no change in Y

X_genre_train, X_genre_test, Y_genre_train, Y_genre_test = train_test_split(X_genre,Y, test_size = .25)


In [25]:
categorical_features  = ["genre"]
categorical_transformer = Pipeline(steps=[("onehot", OneHotEncoder())])

text_features = "message"
text_transformer =Pipeline(
        steps = [ ("corpus",CountVectorizer(analyzer=tokenize)),
           ("tfid", TfidfTransformer())
        ])


preprocessor = ColumnTransformer(
    transformers=[
        ('text', text_transformer, text_features),
        ('cat', categorical_transformer, categorical_features)])

end2end = Pipeline(steps = [("preprocessor",preprocessor),
                            ("classifier", MultiOutputClassifier(MultinomialNB()))])

                   
parameters = {
              "classifier__estimator__alpha"  :[0.1, 0.5, 0.8,1.0 ],
              "preprocessor__text__corpus__analyzer": [tokenize, tokenize2],
              "preprocessor__text__corpus__max_features":[1000, 1500],
              "preprocessor__text__tfid__norm": ["l1", "l2", None],
              "preprocessor__text__tfid__use_idf": [True, False],
            
             }            

In [26]:
cv_expanded = GridSearchCV(end2end, parameters, verbose=1, cv=4)
cv_expanded.fit(X_genre_train, Y_genre_train)

Fitting 4 folds for each of 96 candidates, totalling 384 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.c

c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklear

c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklear

c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklear

c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklear

c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklear

c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklear

c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
c:\python\lib\site-packages\sklear

GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('preprocessor', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('text', Pipeline(memory=None,
     steps=[('corpus', CountVectorizer(analyzer=<function tokenize at 0x09402300>, binary=False,
        decode_erro...fier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'preprocessor__text__tfid__norm': ['l1', 'l2', None], 'preprocessor__text__tfid__use_idf': [True, False], 'preprocessor__text__corpus__max_features': [1000, 1500], 'preprocessor__text__corpus__analyzer': [<function tokenize at 0x09402300>, <function tokenize2 at 0x094023D8>], 'classifier__estimator__alpha': [0.1, 0.5, 0.8, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbos

In [27]:
make_predictions(cv_expanded, X_genre_test, Y_genre_test)



related  

Precision: 0.75158, Recall: 0.84235,  F1 score : 0.79438 ,Accuracy:  0.67165
[[ 665  888]
 [ 358 4643]]

####################


request  

Precision: 0.17164, Recall: 0.2089,  F1 score : 0.18845 ,Accuracy:  0.69774
[[4896  564]
 [ 318  776]]

####################


offer  

Precision: 0.0, Recall: 0.0,  F1 score : 0.0 ,Accuracy:  0.99603
[[6526    0]
 [  28    0]]

####################


aid_related  

Precision: 0.40186, Recall: 0.38688,  F1 score : 0.39423 ,Accuracy:  0.51327
[[3089  759]
 [ 882 1824]]

####################


medical_help  



c:\python\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: 0.078212, Recall: 0.026266,  F1 score : 0.039326 ,Accuracy:  0.89564
[[5969   72]
 [ 406  107]]

####################


medical_products  

Precision: 0.024691, Recall: 0.0063091,  F1 score : 0.01005 ,Accuracy:  0.93988
[[6185   37]
 [ 288   44]]

####################


search_and_rescue  

Precision: 0.0, Recall: 0.0,  F1 score : 0.0 ,Accuracy:  0.9707
[[6359    4]
 [ 188    3]]

####################


security  

Precision: 0.0, Recall: 0.0,  F1 score : 0.0 ,Accuracy:  0.98184
[[6444    0]
 [ 110    0]]

####################


military  

Precision: 0.033784, Recall: 0.023041,  F1 score : 0.027397 ,Accuracy:  0.94583
[[6273   61]
 [ 133   87]]

####################


child_alone  

Precision: 1.0, Recall: 1.0,  F1 score : 1.0 ,Accuracy:  1.0
[[6554]]

####################


water  

Precision: 0.090909, Recall: 0.023364,  F1 score : 0.037175 ,Accuracy:  0.92096
[[6110   30]
 [ 334   80]]

####################


food  

Precision: 0.1171, Recall: 0.067385,  F1 score : 0.085

### 9. Export your model as a pickle file

In [28]:
with open("message_model.pickle", "wb") as pickle_file:
    pickle.dump(cv,pickle_file)

In [29]:
with open("message_genre_model.pickle", "wb") as pickle_file:
    pickle.dump(cv_expanded, pickle_file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.